In [14]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pickle

In [4]:
CIFAR_DIR = '/Users/zhengkaizhang/Desktop/Code/Deep-Learning/Learn/cnn/cifar-10-batches-py/'

dirs = ['batches.meta','data_batch_1','data_batch_2','data_batch_3','data_batch_4','data_batch_5','test_batch']

def unpickle(file):
    with open(file, 'rb') as fo:
        cifar_dict = pickle.load(fo, encoding='bytes')
    return cifar_dict
all_data = []
for direc in dirs:
    all_data.append(unpickle(CIFAR_DIR + direc))

# Helper Function

In [5]:
def one_hot_encoder(vec, vals=10):

    n = len(vec)
    result = np.zeros((n, vals))               
    result[range(n), vec] = 1
    return result

In [75]:
class CifarHelper():
    
    def __init__(self):
        self.i = 0
        self.all_train_batches = all_data[1:6]
        self.test_batch = all_data[6]
        
        self.training_images = None
        self.training_labels = None
        
        self.test_images = None
        self.test_labels = None
    
    def set_up_images(self):
        
        print ("Start to Set up Images and Labels!")
        print ("Training Data First!")

        self.training_images = np.vstack([tmp[b"data"] for tmp in self.all_train_batches])
        train_length = len(self.training_images)
        # Training Image
        self.training_images = self.training_images.reshape(train_length, 3, 32, 32).transpose(0, 2, 3, 1) / 255
        # Training Labels
        self.training_labels = one_hot_encoder(np.hstack([tmp[b"labels"] for tmp in self.all_train_batches]))
        
        print ("Then Test Data.")
        self.test_images = self.test_batch[b'data']
        test_length = len(self.test_images)
        # Test Image
        self.test_images = self.test_images.reshape(test_length, 3, 32, 32).transpose(0, 2, 3, 1) / 255
        # Test Labels
        self.test_labels = one_hot_encoder(self.test_batch[b'labels'])
        
    def next_batch(self, batch_size=50):
        x = self.training_images[self.i:self.i + batch_size].reshape(batch_size, 32, 32, 3)
        y = self.training_labels[self.i:self.i + batch_size]

        self.i = (self.i + batch_size) % len(self.training_images)

        return x, y

In [76]:
ch = CifarHelper()
ch.set_up_images()

Start to Set up Images and Labels!
Training Data First!
Then Test Data.


# Helper Functions

In [73]:
def init_weights(shape):
    weights = tf.truncated_normal(shape=shape, stddev=0.1)
    return tf.Variable(weights)

def init_bias(shape):
    bias = tf.constant(shape=shape, value=1.0)
    return tf.Variable(bias)

def conv2d(x, W):
    return tf.nn.conv2d(input=x, filter=W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2by2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

# Layers

In [72]:
def conv_layer(x, shape):
    W = init_weights(shape)
    b = init_bias([shape[3]])
    return tf.nn.relu(conv2d(x, W) + b)

def fully_connected_layer(x, size):
    x_shape = x.get_shape()
    W = init_weights([int(x_shape[1]), size])
    b = init_bias([size])
    
    return tf.matmul(x, W) + b

# Placeholders

In [82]:
x = tf.placeholder(dtype=tf.float32, shape=[None, 32, 32, 3])
y_true = tf.placeholder(dtype=tf.float32, shape=[None, 10])
hold_prob = tf.placeholder(dtype=tf.float32)

conv1 = conv_layer(x, [4, 4, 3, 32])
pooling1 = max_pool_2by2(conv1)

conv2 = conv_layer(pooling1, [4, 4, 32, 64])
pooling2 = max_pool_2by2(conv2)

flat = tf.reshape(tensor=pooling2, shape=[-1, 8 * 8 * 64])
full_layer_one = tf.nn.relu(fully_connected_layer(x=flat, size=1024))
dropout = tf.nn.dropout(x=full_layer_one, keep_prob=hold_prob)
y_pred = fully_connected_layer(x=dropout, size=10)

loss = tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred)
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
train = optimizer.minimize(loss)

init = tf.global_variables_initializer()

# Train

In [ ]:
steps = 100000
with tf.Session() as sess:
    sess.run(init)

    for step in range(steps):
        x_batch, y_batch = ch.next_batch()
        sess.run(train, feed_dict={x:x_batch, y_true:y_batch, hold_prob:0.5})

        if (step % 100) != 0:
            continue
        
        x_test = ch.test_images
        y_test = ch.test_labels
        print ("At Step {}".format(step))
        matches = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y_true, 1))
        acc = tf.reduce_mean(tf.cast(matches, dtype=tf.float32))
        print ("Accuracy is ")
        print (sess.run(acc, feed_dict={x:x_test, y_true:y_test, hold_prob:1.0}))
        print ()

At Step 0
Accuracy is 
0.1

At Step 100
Accuracy is 
